In [1]:
using Mocha

01-jun 02:14:28:INFO:root:Configuring Mocha...
01-jun 02:14:28:INFO:root: * CUDA       disabled by default
01-jun 02:14:28:INFO:root: * Native Ext disabled by default
01-jun 02:14:28:INFO:root:Mocha configured, continue loading module...


In [9]:
data_layer  = HDF5DataLayer(name="train-data",source="train.txt",batch_size=64,shuffle=false)


HDF5DataLayer(train-data)

In [10]:
conv_layer=ConvolutionLayer(name="conv1",n_filter=20,kernel=(5,5),bottoms=[:data], tops=[:conv])

ConvolutionLayer(conv1)

In [11]:
pool_layer = PoolingLayer(name="pool1", kernel=(2,2), stride=(2,2),
    bottoms=[:conv], tops=[:pool])
conv2_layer = ConvolutionLayer(name="conv2", n_filter=50, kernel=(5,5),
    bottoms=[:pool], tops=[:conv2])
pool2_layer = PoolingLayer(name="pool2", kernel=(2,2), stride=(2,2),
    bottoms=[:conv2], tops=[:pool2])

PoolingLayer(pool2)

In [12]:
fc1_layer  = InnerProductLayer(name="ip1", output_dim=500,
    neuron=Neurons.ReLU(), bottoms=[:pool2], tops=[:ip1])
fc2_layer  = InnerProductLayer(name="ip2", output_dim=1,
    bottoms=[:ip1], tops=[:ip2])

InnerProductLayer(ip2)

In [13]:
#loss_layer = SquareLossLayer(name="loss", bottoms=[:ip2,:label])
loss_layer = SquareLossLayer(name="loss", bottoms=[:ip2,:label])


SquareLossLayer(loss)

In [14]:
backend = CPUBackend()


CPUBackend

In [15]:
common_layers = [conv_layer, pool_layer, conv2_layer, pool2_layer,
    fc1_layer, fc2_layer]
net = Net("test", backend, [data_layer, common_layers..., loss_layer])

01-jun 02:17:01:INFO:root:Constructing net test on CPUBackend...
01-jun 02:17:01:INFO:root:Topological sorting 8 layers...
01-jun 02:17:01:INFO:root:Setup layers...


LoadError: `size` has no method matching size(::HDF5Group)
while loading In[15], in expression starting on line 3

In [77]:
exp_dir = "snapshots"

params = SolverParameters(max_iter=100, regu_coef=0.0005,
    mom_policy=MomPolicy.Fixed(0.9),
    lr_policy=LRPolicy.Inv(0.01, 0.0001, 0.75),
    load_from=exp_dir)
solver = SGD(params)

SGD(SolverParameters(Inv(0.01,0.0001,0.75),Fixed(0.9),100,0.0005,"snapshots"),CoffeeLounge("",1,:merge,Dict{String,Dict{Int64,FloatingPoint}}(),[],true,97199648,53052248))

In [ ]:
setup_coffee_lounge(solver, save_into="$exp_dir/statistics.hdf5", every_n_iter=10)


In [ ]:
add_coffee_break(solver, TrainingSummary(), every_n_iter=100)


In [ ]:
data_layer_test = HDF5DataLayer(name="test-data", source="test.txt", batch_size=100)
acc_layer = AccuracyLayer(name="test-accuracy", bottoms=[:ip2, :y])
test_net = Net("MNIST-test", backend, [data_layer_test, common_layers..., acc_layer])

In [ ]:
add_coffee_break(solver, ValidationPerformance(test_net), every_n_iter=1000)


In [ ]:
solve(solver, net)


In [ ]:

destroy(net)
destroy(test_net)
shutdown(backend)